In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import NullFormatter
import psycopg2
import matplotlib.ticker as mtick
import re

In [11]:
cars = pd.read_csv("/Users/tingjue/170 project/cars_revised.csv",index_col=0)

In [21]:
cars

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,paint_color,description,state,id
0,salt lake city,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,63500.0,clean,manual,wvwpf7aj6cw316713,4wd,black,price reduced! -garage kept -low miles (63k)...,ut,0
1,salt lake city,4600,2008.0,honda,civic,good,4 cylinders,gas,110982.0,clean,automatic,2hgfa16528h503005,fwd,grey,selling this 2008 honda civic 1.8l four door l...,ut,1
2,salt lake city,3995,1999.0,subaru,outback,good,4 cylinders,gas,215349.0,clean,automatic,4s3bg6859x6660102,4wd,blue,"1999 subaru outback ltd 30th -- $3,995 â...",ut,2
3,salt lake city,18999,2015.0,mercedes_benz,cla-class,excellent,4 cylinders,gas,37000.0,clean,automatic,wddsj4eb3fn168594,fwd,white,www.elegantautosales.netcall â 916â794â3...,ut,3
4,salt lake city,18999,2013.0,bmw,3 series,excellent,6 cylinders,gas,65000.0,clean,automatic,wbakg1c59dj217447,rwd,black,www.elegantautosales.netcall â 916â794â3...,ut,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75626,salt lake city,33895,2016.0,ford,f-150,good,6 cylinders,gas,49698.0,clean,automatic,1ftfw1eg9gfa33106,4wd,blue,"2016 ford f-150 xlt -- $33,895 â call us ...",ut,83063
75627,salt lake city,6500,2008.0,chevrolet,express 3500,good,8 cylinders,gas,195000.0,clean,automatic,1gahg39k981133844,rwd,white,"this is a 15 passenger van, its been a valued ...",ut,83064
75628,salt lake city,7500,2014.0,nissan,versa note,good,4 cylinders,gas,99990.0,clean,manual,3n1ce2cp2el410458,fwd,white,"this car is going strong, i have always been o...",ut,83065
75629,salt lake city,32000,2018.0,gmc,canyon crew cab slt,excellent,6 cylinders,gas,39457.0,clean,automatic,1gtg6denxj1127455,4wd,black,really nice gmc canyon slt 4x4 black on black ...,ut,83066


In [12]:
edmunds = pd.read_csv("/Users/tingjue/170 project/edmunds review.csv",index_col=0)

In [13]:
edmunds=edmunds.rename(columns={'id': 'id2'})

In [14]:
edmunds = edmunds.append({'model':'not matched','id2':4646}, ignore_index=True)

In [33]:
edmunds.head(10)

,year,manufacturer,model,avg_rating,count,id2
0,1997.0,acura,cl,4.607759,58.0,0
1,1998.0,acura,cl,4.678977,44.0,1
2,1999.0,acura,cl,4.666667,54.0,2
3,2001.0,acura,cl,4.573864,110.0,3
4,2002.0,acura,cl,4.442857,35.0,4
5,2003.0,acura,cl,4.568584,113.0,5
6,2013.0,acura,ilx,4.703125,16.0,6
7,2014.0,acura,ilx,4.538462,13.0,7
8,2015.0,acura,ilx,4.437500,4.0,8
9,2016.0,acura,ilx,4.117188,32.0,9


In [9]:
#general matched algorithm 
def match(manuf):
    matched=dict()
    #df = cars_sample[cars_sample.manufacturer==manuf]
    df = cars[cars.manufacturer==manuf]
    for indexc, rowc in df.iterrows():
        current_score = 0
        current_id = None
        manu=rowc['manufacturer']
        for indexe,rowe in edmunds[edmunds.manufacturer==manu].iterrows():
            temp_score = 0
            temp_id = rowe['id2']
            if rowc['year']==rowe['year']:
                temp_score+=5
            modelc = re.split('\*|-|\(|\s|\)|\+|,',rowc['model'])
            modelc = [x for x in modelc if x != '']
            modele= rowe['model'].split()

            for i in modelc:
                for e in modele:
                    if i == e:
                        temp_score += 10
            if temp_score > current_score:
                current_score = temp_score
                current_id = temp_id
        if current_score < 6:
            matched[rowc['id']]=4646
        else:
            matched[rowc['id']]=current_id
    return matched 



In [10]:
# algorithm for bmw 
def match_bmw(manuf):
    matched=dict()
    #df = cars_sample[cars_sample.manufacturer==manuf]
    df = cars[cars.manufacturer==manuf]


    for indexc, rowc in df.iterrows():
        current_score = 0
        current_id = None
        manu=rowc['manufacturer']
        for indexe,rowe in edmunds[edmunds.manufacturer==manu].iterrows():
            temp_score = 0
            temp_id = rowe['id2']
            if rowc['year']==rowe['year']:
                temp_score+=5
            model = re.split('\*|-|\(|\s|\)|\+|,',rowc['model'])
            model = [x for x in model if x != '']
            for i in model:
                if i in rowe['model']:
                    temp_score += 10
                else:
                    m = model[0][0]
                    if m in rowe['model']:
                        temp_score += 10
                        
            if temp_score > current_score:
                current_score = temp_score
                current_id = temp_id
        if current_score < 6:
            matched[rowc['id']]=4646
        else:
            matched[rowc['id']]=current_id
    return matched 

In [11]:
def match_cadillac(manuf):
    matched=dict()
    #df = cars_sample[cars_sample.manufacturer==manuf]
    df = cars[cars.manufacturer==manuf]


    for indexc, rowc in df.iterrows():
        current_score = 0
        current_id = None
        manu=rowc['manufacturer']
        for indexe,rowe in edmunds[edmunds.manufacturer==manu].iterrows():
            temp_score = 0
            temp_id = rowe['id2']
            if rowc['year']==rowe['year']:
                temp_score+=5
            modelc = re.split('\*|-|\(|\s|\)|\+|,',rowc['model'])
            modelc = [x for x in modelc if x != '']
            modele= rowe['model'].split()



            for i in modelc:
                for e in modele:
                    if i == e:
                        temp_score += 10
                    else:
                        if len(modelc[0])>=2:
                            m = modelc[0][0]+modelc[0][1]
                            if e.startswith(m)== True:
                                temp_score += 10
                        
            if temp_score > current_score:
                current_score = temp_score
                current_id = temp_id
        if current_score < 6:
            matched[rowc['id']]=4646
        else:
            matched[rowc['id']]=current_id
    return matched

In [222]:
l = match_cadillac('lexus')

In [238]:
test = cars_sample[cars_sample.manufacturer=='lexus']
test['id2']=l.values()
t = pd.merge(test, edmunds, on='id2')


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
models = edmunds[edmunds.manufacturer == 'toyota']
set(models['model'])

{'4runner',
 '86',
 'avalon',
 'avalon hybrid',
 'c-hr',
 'camry',
 'camry hybrid',
 'celica',
 'corolla',
 'echo',
 'fj',
 'highlander',
 'highlander hybrid',
 'land',
 'matrix',
 'mirai',
 'mr2',
 'paseo',
 'previa',
 'prius',
 'rav4',
 'rav4 hybrid',
 'sequoia',
 'sienna',
 'supra',
 'tercel',
 'venza',
 'yaris'}

In [14]:
pd.set_option('display.max_rows', None)

In [25]:
b = temp[temp.manufacturer_x=='volvo']
#b.sample(n=200)
b

,year_x,manufacturer_x,model_x,id,ind,year_y,model_y,avg_rating,count
price,,,,,,,,,
7500,1988.0,volvo,245,17590,NaN,NaN,NaN,NaN,NaN
1500,1985.0,volvo,245 dl,17592,NaN,NaN,NaN,NaN,NaN
2600,1990.0,volvo,245,19816,NaN,NaN,NaN,NaN,NaN
1450,1987.0,volvo,240 wagon,20839,NaN,NaN,NaN,NaN,NaN
3200,1992.0,volvo,240,78948,NaN,NaN,NaN,NaN,NaN
4250,1995.0,volvo,850t5r,84862,NaN,NaN,NaN,NaN,NaN
6495,1993.0,volvo,240 sedan,91469,NaN,NaN,NaN,NaN,NaN
1400,1987.0,volvo,240 wagon,92253,NaN,NaN,NaN,NaN,NaN
3850,1985.0,volvo,240,100326,NaN,NaN,NaN,NaN,NaN


In [226]:
list = ['acura', #drop nan
'audi', #drop nan
 'bmw', #match_bmw('bmw')
 'buick',#
 'cadillac', #match_cadillac('cadillac')
 'chevrolet', #
 'chrysler', #
 'dodge', #
 'fiat', #
 'ford', #   f150,f250,f350,f450 
 'gmc', # match_cadillac('gmc')
 'honda', #
 'hyundai', #
 'infiniti', #
 'jaguar', #match_bmw('infiniti')
 'jeep',#
 'kia',#
 'land_rover', #
 'lexus', # match_cadillac('lexus')
 'lincoln', #
 'mazda',#
 'mercedes_benz', #
 'mercury',#
 'mini',# 
# __________
 'mitsubishi',#
 'nissan', # 少frontier 
 'pontiac', #
 'ram', #少了1500 和3500 
 'subaru', #
 'toyota', #少 tundra，tacoma
 'volkswagen',
 'volvo'] #

In [279]:
l1 = ['acura', #drop nan
'audi', #drop nan
 'buick',#
 'chevrolet', #
 'chrysler', #
 'dodge', #
 'fiat', #
 'honda', #
 'hyundai', #
 'infiniti', #
 'jeep',#
 'kia',#
 'land_rover', #
 'lincoln', #
 'mazda',#
 'mercedes_benz', #
 'mercury',#
 'mini']# 

l2 = [
 'bmw', #match_bmw('bmw')
 'jaguar', #match_bmw('infiniti')
]

l3 = [ 'cadillac','gmc','lexus']
    

In [280]:
match_id = dict()
for i in l1:
    match_id.update(match(i))
for i in l2:
    match_id.update(match_bmw(i))


In [281]:
for i in l3:
    match_id.update(match_cadillac(i))

In [282]:
a = pd.DataFrame.from_dict(match_id,orient='index')

In [283]:
a.to_csv("match_id.csv")

In [27]:
l4 = [  
'mitsubishi',
 'pontiac', 
 'subaru', 
 'volkswagen',
 'volvo'] 
match_id1 = dict()
for i in l4:
    match_id1.update(match(i))


In [28]:
b = pd.DataFrame.from_dict(match_id1,orient='index')
b.to_csv("match_id1.csv")

In [34]:
l5 = ['ford','nissan','ram','toyota']
match_id2 = dict()
for i in l5:
    match_id2.update(match(i))
    

In [35]:
c = pd.DataFrame.from_dict(match_id2,orient='index')
c.to_csv("match_id2.csv")

# merge datasets

In [15]:
matched = pd.read_csv("/Users/tingjue/170 project/match_id.csv",index_col=0)

In [16]:
cars2 = pd.merge(cars, matched, on='id')
final_cars = pd.merge(cars2,edmunds,on='id2')


In [17]:
final_cars = final_cars.drop(['year_y'], axis = 1)\
.drop(['id2'], axis = 1)\
.drop(['manufacturer_y'],axis = 1)

In [41]:
final_cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80446 entries, 0 to 80445
Data columns (total 20 columns):
region            80446 non-null object
price             80446 non-null int64
year_x            80446 non-null float64
manufacturer_x    80446 non-null object
model_x           80446 non-null object
condition         80446 non-null object
cylinders         80446 non-null object
fuel              80446 non-null object
odometer          80446 non-null float64
title_status      80446 non-null object
transmission      80446 non-null object
vin               80446 non-null object
drive             80446 non-null object
paint_color       80446 non-null object
description       80446 non-null object
state             80446 non-null object
id                80446 non-null int64
model_y           80446 non-null object
avg_rating        74003 non-null float64
count             74003 non-null float64
dtypes: float64(4), int64(2), object(14)
memory usage: 12.9+ MB


In [18]:
is_NaN = final_cars.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = final_cars[row_has_NaN] 

In [19]:
t = final_cars.dropna(axis=0, how='any', thresh=None, 
                   subset=None, 
                   inplace=False).reset_index(drop=True)

In [20]:
t = t[t.model_y != 'not matched']

In [21]:
t.info() #13000 not matched

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67560 entries, 0 to 74002
Data columns (total 20 columns):
region            67560 non-null object
price             67560 non-null int64
year_x            67560 non-null float64
manufacturer_x    67560 non-null object
model_x           67560 non-null object
condition         67560 non-null object
cylinders         67560 non-null object
fuel              67560 non-null object
odometer          67560 non-null float64
title_status      67560 non-null object
transmission      67560 non-null object
vin               67560 non-null object
drive             67560 non-null object
paint_color       67560 non-null object
description       67560 non-null object
state             67560 non-null object
id                67560 non-null int64
model_y           67560 non-null object
avg_rating        67560 non-null float64
count             67560 non-null float64
dtypes: float64(4), int64(2), object(14)
memory usage: 10.8+ MB


In [22]:
t.to_csv("cars1.0.csv")

In [23]:
df = pd.read_csv("/Users/tingjue/170 project/cars1.0.csv",index_col=0)

In [24]:
df.head()

,region,price,year_x,manufacturer_x,model_x,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,paint_color,description,state,id,model_y,avg_rating,count
0,salt lake city,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,63500.0,clean,manual,wvwpf7aj6cw316713,4wd,black,price reduced! -garage kept -low miles (63k)...,ut,0,golf,4.600694,36.0
1,wenatchee,9891,2012.0,volkswagen,golf tdi,excellent,4 cylinders,diesel,89419.0,clean,automatic,wvwdm7aj0cw181950,fwd,white,_volkswagen_golf_energy auto sales --- call us...,wa,4398,golf,4.600694,36.0
2,new haven,7995,2012.0,volkswagen,golf,good,5 cylinders,gas,41600.0,clean,automatic,wvwdb7ajxcw354699,fwd,grey,2012 volkswagen_ golf_ offered by...,ct,23723,golf,4.600694,36.0
3,ft myers / sw florida,12900,2012.0,volkswagen,golf,excellent,4 cylinders,diesel,68073.0,clean,automatic,wvwdm7aj9cw040181,fwd,blue,2012 volkswagen golf tdi - (blue graphite meta...,fl,25205,golf,4.600694,36.0
4,chicago,21270,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,53169.0,clean,automatic,wvwpf7aj3cw335770,4wd,blue,2012 volkswagen golf r 4dr hb w/sunroof & navi...,il,33832,golf,4.600694,36.0


In [26]:
df = df.rename(columns={'year_x': 'year','manufacturer_x':'manufacturer','model_y':'model'})

In [27]:
df.head()

,region,price,year,manufacturer,model_x,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,paint_color,description,state,id,model,avg_rating,count
0,salt lake city,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,63500.0,clean,manual,wvwpf7aj6cw316713,4wd,black,price reduced! -garage kept -low miles (63k)...,ut,0,golf,4.600694,36.0
1,wenatchee,9891,2012.0,volkswagen,golf tdi,excellent,4 cylinders,diesel,89419.0,clean,automatic,wvwdm7aj0cw181950,fwd,white,_volkswagen_golf_energy auto sales --- call us...,wa,4398,golf,4.600694,36.0
2,new haven,7995,2012.0,volkswagen,golf,good,5 cylinders,gas,41600.0,clean,automatic,wvwdb7ajxcw354699,fwd,grey,2012 volkswagen_ golf_ offered by...,ct,23723,golf,4.600694,36.0
3,ft myers / sw florida,12900,2012.0,volkswagen,golf,excellent,4 cylinders,diesel,68073.0,clean,automatic,wvwdm7aj9cw040181,fwd,blue,2012 volkswagen golf tdi - (blue graphite meta...,fl,25205,golf,4.600694,36.0
4,chicago,21270,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,53169.0,clean,automatic,wvwpf7aj3cw335770,4wd,blue,2012 volkswagen golf r 4dr hb w/sunroof & navi...,il,33832,golf,4.600694,36.0


In [28]:
cols = df.columns.tolist()

In [29]:
cols

['region',
 'price',
 'year',
 'manufacturer',
 'model_x',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'vin',
 'drive',
 'paint_color',
 'description',
 'state',
 'id',
 'model',
 'avg_rating',
 'count']

In [32]:
l = ['id',
 'vin',
 'price',
 'year',
 'manufacturer',
 'model_x',
 'model',
 'avg_rating',
 'count',
 'condition',
 'cylinders',
 'fuel',
 'odometer',
 'title_status',
 'transmission',
 'drive',
 'paint_color',
 'description',
 'region',
 'state',
]

In [33]:
df = df[l]

In [34]:
df.head()

,id,vin,price,year,manufacturer,model_x,model,avg_rating,count,condition,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,description,region,state
0,0,wvwpf7aj6cw316713,17899,2012.0,volkswagen,golf r,golf,4.600694,36.0,excellent,4 cylinders,gas,63500.0,clean,manual,4wd,black,price reduced! -garage kept -low miles (63k)...,salt lake city,ut
1,4398,wvwdm7aj0cw181950,9891,2012.0,volkswagen,golf tdi,golf,4.600694,36.0,excellent,4 cylinders,diesel,89419.0,clean,automatic,fwd,white,_volkswagen_golf_energy auto sales --- call us...,wenatchee,wa
2,23723,wvwdb7ajxcw354699,7995,2012.0,volkswagen,golf,golf,4.600694,36.0,good,5 cylinders,gas,41600.0,clean,automatic,fwd,grey,2012 volkswagen_ golf_ offered by...,new haven,ct
3,25205,wvwdm7aj9cw040181,12900,2012.0,volkswagen,golf,golf,4.600694,36.0,excellent,4 cylinders,diesel,68073.0,clean,automatic,fwd,blue,2012 volkswagen golf tdi - (blue graphite meta...,ft myers / sw florida,fl
4,33832,wvwpf7aj3cw335770,21270,2012.0,volkswagen,golf r,golf,4.600694,36.0,excellent,4 cylinders,gas,53169.0,clean,automatic,4wd,blue,2012 volkswagen golf r 4dr hb w/sunroof & navi...,chicago,il


In [42]:
c = df[df.duplicated(subset=['vin'],keep='first')]

In [41]:
df = df[df.vin != '00000000000000000']
df = df[df.vin != '12345678901234567']

In [43]:
c.sort_values(by=['vin'])

,id,vin,price,year,manufacturer,model_x,model,avg_rating,count,condition,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,description,region,state
62108,52931,1456789jkytd45678,4988,2002.0,ford,f150 supercrew cab xlt,f150 f 150,4.458101,179.0,excellent,8 cylinders,gas,183888.0,clean,automatic,rwd,white,2002 ford f150 crew cab pickup extra nice pow...,st joseph,mo
62097,40731,1456789jkytd45678,4988,2002.0,ford,f150 supercrew cab,f150 f 150,4.458101,179.0,excellent,8 cylinders,gas,183888.0,clean,automatic,rwd,white,2002 ford f150 super crew cab pickup extra nic...,topeka,ks
67694,21541,19uua568x3a081985,1800,2003.0,acura,tl type s,tl,4.440029,173.0,excellent,6 cylinders,gas,199632.0,clean,automatic,fwd,white,2003 tl type s in (mostly) good condition. the...,denver,co
46195,74423,19uua66287a027883,7990,2007.0,acura,tl,tl,4.601648,182.0,like new,6 cylinders,gas,77679.0,clean,automatic,fwd,grey,"2007 acura tl 3.2tl with nav. system - $7,990 ...",rhode island,ri
68404,52002,19uua8f21ca039203,10995,2012.0,acura,tl base 4dr sedan,tl,4.334184,49.0,good,6 cylinders,gas,112146.0,clean,automatic,4wd,silver,why buy from taylor auto sales?we have been wo...,joplin,mo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43318,57585,zfberfat7g6b17417,9995,2016.0,ram,promaster city,promaster,2.771429,35.0,good,4 cylinders,gas,111622.0,clean,automatic,fwd,white,2016 ram promaster city vehicle informatio...,las vegas,nv
43322,60061,zfberfatxg6c55940,12950,2016.0,ram,promaster city,promaster,2.771429,35.0,good,4 cylinders,gas,76292.0,clean,automatic,fwd,white,dkr incask for: paul or randyââ (817) 239-73...,roswell / carlsbad,nm
43324,63618,zfberfatxg6c55940,12950,2016.0,ram,promaster city,promaster,2.771429,35.0,good,4 cylinders,gas,76292.0,clean,automatic,fwd,white,dkr incask for: paul or randyââ (817) 239-73...,bismarck,nd
43309,38503,zfberfbt1g6b11269,15900,2016.0,ram,promaster city cargo van,promaster,2.771429,35.0,excellent,4 cylinders,gas,61976.0,clean,automatic,fwd,white,"we buy and sell a lot of high quality, high va...",des moines,ia


In [44]:
df[df.vin=='19uua8f21ca039203']

,id,vin,price,year,manufacturer,model_x,model,avg_rating,count,condition,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,description,region,state
68392,12408,19uua8f21ca039203,10995,2012.0,acura,tl base 4dr sedan,tl,4.334184,49.0,good,6 cylinders,gas,112146.0,clean,automatic,4wd,silver,why buy from taylor auto sales?we have been wo...,fayetteville,ar
68404,52002,19uua8f21ca039203,10995,2012.0,acura,tl base 4dr sedan,tl,4.334184,49.0,good,6 cylinders,gas,112146.0,clean,automatic,4wd,silver,why buy from taylor auto sales?we have been wo...,joplin,mo


In [53]:
df = df.drop_duplicates(subset='vin', keep="first")

In [54]:
df.to_csv('cars2.0.csv')

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59917 entries, 0 to 74002
Data columns (total 20 columns):
id              59917 non-null int64
vin             59917 non-null object
price           59917 non-null int64
year            59917 non-null float64
manufacturer    59917 non-null object
model_x         59917 non-null object
model           59917 non-null object
avg_rating      59917 non-null float64
count           59917 non-null float64
condition       59917 non-null object
cylinders       59917 non-null object
fuel            59917 non-null object
odometer        59917 non-null float64
title_status    59917 non-null object
transmission    59917 non-null object
drive           59917 non-null object
paint_color     59917 non-null object
description     59917 non-null object
region          59917 non-null object
state           59917 non-null object
dtypes: float64(4), int64(2), object(14)
memory usage: 9.6+ MB


In [8]:
see = pd.read_csv("/Users/tingjue/170 project/cars2.0.csv",index_col=0)

In [9]:
see.sample(10)

,id,vin,price,year,manufacturer,model_x,model,avg_rating,count,condition,cylinders,fuel,odometer,title_status,transmission,drive,paint_color,description,region,state
40056,61219,yv1rh592x62500756,4500,2006.0,volvo,s60,s60,4.596774,93.0,like new,5 cylinders,gas,73000.0,clean,automatic,4wd,blue,second owner 2006 volvo s60 awd 5 cyl 2.5 lite...,hudson valley,ny
52676,31435,shhfk7h69ku222514,24995,2019.0,honda,civic,civic,4.617880,158.0,excellent,4 cylinders,gas,3627.0,clean,automatic,fwd,grey,"pacific honda 188 s. beretania st. honolulu, h...",hawaii,hi
36743,55270,wbapk7g59bnn70648,12395,2011.0,bmw,328xdrive,3 series,4.195388,103.0,excellent,6 cylinders,gas,75392.0,clean,automatic,4wd,grey,2011 bmw 328xdrive w/ only 75k miles and loade...,greensboro,nc
26680,35517,2gcek19b661323282,8980,2006.0,chevrolet,silverado 1500,silverado,4.043750,20.0,good,8 cylinders,gas,153376.0,clean,automatic,4wd,white,just arrived!!! 2006 chevy silverado 1500 ext...,"st louis, mo",il
69256,43596,5tehn72n23z201103,7700,2003.0,toyota,tacoma,tacoma,4.413889,135.0,good,6 cylinders,gas,192000.0,clean,automatic,4wd,green,"2003 toyota tacoma sr5, double cab, 3.4l v6, 4...",maine,me
24887,10930,1ftfw1et6dkf68135,23900,2013.0,ford,f-150,f150 f 150,4.275862,58.0,good,6 cylinders,gas,93409.0,clean,automatic,4wd,silver,"2013 *ford* *f-150* fx4 truck - $23,900call us...",phoenix,az
11731,64267,2gcek13c481132450,13480,2008.0,chevrolet,silverado 1500,silverado,4.000000,31.0,excellent,8 cylinders,gas,93650.0,clean,automatic,4wd,black,"*blackout wheels, bluetooth,chrome step rails,...",akron / canton,oh
55784,46381,2gkflte3xh6229640,19600,2017.0,gmc,terrain sle-2 sport utility,terrain,3.277778,18.0,good,6 cylinders,gas,41082.0,clean,other,4wd,white,vehicle stock# 2000421188ð± you can immediat...,western massachusetts,ma
66887,44932,km8ju3ag6fu955758,12495,2015.0,hyundai,tucson,tucson,3.461538,13.0,excellent,4 cylinders,gas,68670.0,clean,automatic,fwd,black,drive it home today. call (or text) us now !!c...,frederick,md
43777,46567,5n1at2mv9ec865810,15925,2014.0,nissan,rogue,rogue,3.777778,63.0,excellent,4 cylinders,gas,59660.0,clean,automatic,4wd,red,2014 nissan rogue sl - (cayenne red) bad c...,worcester / central ma,ma
